# GABARITO

-----------PARTE II (3/3)--------------

    Extração das features. 
    PCA das features.
    Classificação.

-----------PARTE III (2/2)-------------

    Cálculo das métricas de classificação.
    Relatório. 

-----------PARTE IV (2/2)--------------

    Breve introdução
    Separar em tóp.
    Git hub (atualizar o git com o ep3)
    Cabeçalho dos int
    Comentar cód.
    Organização do código e notebook 
    Read-me (pode ser o read-me do git atualizado)

NOTA (10/10)

# MAC0417/5768 - Visão e Processamento de Imagens (2021)

Exercício Programa 3.2 - Classificação de imagens

Gabriela Melo e Richard Block

Projeto GitHub: https://github.com/gabi-melo/image_processing/tree/main/EP3

Base de imagens: https://www.kaggle.com/gabrielamelo/image-processing


# Resumo

Nessa parte do EP realizamos a etapa de classificação, baseada nas imagens binárias com Feret Box obtidas na primeira parte. As imagens foram redimensionaras para que todas as classes apresentem o mesmo tamanho. Os dados foram separados em conjuntos de treino e de teste. Então foi executada uma etapa de preparação para o aprendizado, com a extração do vetor de características. Para isso, foi utilizada a análise de componentes principais (PCA). A PCA descobre os eixos/vetores de maior variância dos dados, fazendo uma análise a partir da matriz de covariância. O vetor que indica a maior variabilidade dos dados é o primeiro autovetor. O vetor perpendicular é o segundo autovetor, indicando o segundo vetor de maior variabilidade. A transformada PCA muda a coordenada dos pontos definida pelos autovetores, realizando a redução da dimensionalidade.

O vetor de características resultante foi então submetido à classificação, utilizando um algoritmo de máquina de suporte vetorial (SVM). Um classificador define um modelo algébrico para qual é a superfície de decisão. O modelo mais simples é uma linha reta que divide o conjunto de dados para fazer a separação dos dados entre as diferentes classes. As máquinas de suporte vetorial procuram criar uma reta que define uma função critério, a qual é otimizada. O procedimento de otimização faz calcula qual é a linha reta que maximiza as distâncias dos pontos mais próximos da reta. O processo de treinamento de SVM descobre qual é essa linha reta que apresenta a maior distância possível de todas as classes. Esse treino foi realizado exclusivamente nos dados do conjunto de treino.
Foram calculadas métricas de avaliação da qualidade do modelo nos dados do conjunto de teste. Foi realizada a predição dos rótulos das classes do conjunto de teste, e foram computadas variáveis da performance do classificador.

# Download do Kaggle dataset

In [39]:
from glob import glob 
import cv2
import os
from PIL import Image
from skimage import io

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [13]:
# changing the working directory
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [14]:
!kaggle datasets download -d gabrielamelo/image-processing

image-processing.zip: Skipping, found more recently modified local copy (use --force to force download)


In [37]:
# check content
!ls /content

gdrive	sample_data


In [58]:
# unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  image-processing.zip
replace augmentedDataset/augmentedDataset/caneta_1_dia_dentro_madeira_1_conv.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Classificação usando SVM

## As imagens já foram pré-processadas, todas padronizadas no tamanho (64, 64).

### Podemos visualizar algumas das imagens.

In [25]:
from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [60]:
# objects = fetch_lfw_people(data_home = 'path', download_if_missing=False) #carregar imagens

In [ ]:
# fig = plt.figure(figsize=(8, 6))

# for i in range(15):
#     ax = fig.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
#     ax.imshow(objects[i], cmap=plt.cm.bone)

### Divisão da amostra em dados de treinamento e dados de teste

In [52]:
paths = glob('/*.jpg') # path para os objects

X = []

for path in paths:

  an_image = Image.open(path)
  image_sequence = an_image.getdata()
  image_array = np.array(image_sequence)
  X.append(image_array)

X = np.array(X)
print(X)

y = []

for path in paths:
    
  img_class = path.split('/')[-1].split('_')[0]
  y.append(img_class)

y = np.array(y)
print(y)

[[  2   3   4 ... 164 164 163]
 [166 166 166 ... 126 108  97]
 [ 79  79  79 ... 124 124 124]
 ...
 [  0   0   0 ...   0   0   0]
 [ 11  25  30 ...  57  43  59]
 [ 54  54  54 ...  56  56  56]]
['chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave'
 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave'
 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'chave'
 'chave' 'chave' 'chave' 'chave' 'chave' 'chave' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta' 'caneta'
 'caneta' '

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(X_train.shape, X_test.shape)

(150, 480000) (50, 480000)


### PCA

In [54]:
pca = PCA(n_components=150, whiten=True)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=150, random_state=None,
    svd_solver='auto', tol=0.0, whiten=True)

Um exemplo de objeto "médio"

In [ ]:
plt.imshow(pca.mean_.reshape(faces.images[0].shape),
           cmap=plt.cm.bone)

Os desvios em relação à média ao longo dos eixos ortogonais.

In [ ]:
print(pca.components_.shape)

### Visualização dos componentes principais, ordenados conforme importância.

In [ ]:
fig = plt.figure(figsize=(16, 6))
for i in range(30):
    ax = fig.add_subplot(3, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(pca.components_[i].reshape(faces.images[0].shape),
              cmap=plt.cm.bone)

Os dados de treinamento e de teste projetados nas bases do PCA.

In [55]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(X_train_pca.shape)

(150, 150)


In [56]:
print(X_test_pca.shape)

(50, 150)


## SVM

In [61]:
clf = svm.SVC(C=5., gamma=0.001)
clf.fit(X_train_pca, y_train)

### Visualização de alguns resultados: imagens com os rótulos aprendidos.

In [ ]:
import numpy as np
fig = plt.figure(figsize=(8, 6))
for i in range(15):
    ax = fig.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(X_test[i].reshape(faces.images[0].shape),
              cmap=plt.cm.bone)
    y_pred = clf.predict(X_test_pca[i, np.newaxis])[0]
    color = ('black' if y_pred == y_test[i] else 'red')
    ax.set_title(y_pred, fontsize='small', color=color)

## Métricas de classificação

### Precisão, Recall, F1-Score e Suporte

In [ ]:
y_pred = clf.predict(X_test_pca)
print(classification_report(y_test, y_pred))

### Confusion-matrix
Um classificador perfeito apresenta números diferentes de zero somente na diagonal principal, e zero no restante.

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays
# base de ROIs

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)


# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

# X = conjunto de dados/imagens
# y = conjunto de rótulos


# #############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction

n_components = 150

# calcula os autovetores/extração dos vetores de características
print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

# calcula as dimensões da transformada de PCA
print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))


# #############################################################################
# Train a SVM classification model

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(X_train_pca, y_train) # ajuste da função critério
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)


# #############################################################################
# Quantitative evaluation of the model quality on the test set

# predição dos rótulos do test set

print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

# variáveis da performance do classificador
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))


# #############################################################################
# Qualitative evaluation of the predictions using matplotlib

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()


# precisão da acurácia da classificação
# revocação captura, da amostra inteira, quantos objetos foram possíveis recuperar de cada classe
# f1 
# suporte

# matriz de confusão coloca todas as classes nas linhas e colunas e mostra o que 
# foi classificado como o que